Faster R-CNN model connected with Anvil Application

In [7]:
import torchvision
print(torchvision.__version__)


0.17.1


In [10]:
# Load the checkpoint file and print its keys
checkpoint = torch.load('/Users/kimberleycollins/Desktop/Thesis/Thesis/Prototype/Application/model_final.pth', map_location=torch.device('cpu'))
print(checkpoint.keys())


odict_keys(['backbone.body.0.0.weight', 'backbone.body.0.1.weight', 'backbone.body.0.1.bias', 'backbone.body.0.1.running_mean', 'backbone.body.0.1.running_var', 'backbone.body.1.block.0.0.weight', 'backbone.body.1.block.0.1.weight', 'backbone.body.1.block.0.1.bias', 'backbone.body.1.block.0.1.running_mean', 'backbone.body.1.block.0.1.running_var', 'backbone.body.1.block.1.0.weight', 'backbone.body.1.block.1.1.weight', 'backbone.body.1.block.1.1.bias', 'backbone.body.1.block.1.1.running_mean', 'backbone.body.1.block.1.1.running_var', 'backbone.body.2.block.0.0.weight', 'backbone.body.2.block.0.1.weight', 'backbone.body.2.block.0.1.bias', 'backbone.body.2.block.0.1.running_mean', 'backbone.body.2.block.0.1.running_var', 'backbone.body.2.block.1.0.weight', 'backbone.body.2.block.1.1.weight', 'backbone.body.2.block.1.1.bias', 'backbone.body.2.block.1.1.running_mean', 'backbone.body.2.block.1.1.running_var', 'backbone.body.2.block.2.0.weight', 'backbone.body.2.block.2.1.weight', 'backbone.b

In [16]:
# Check the current version
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

# If needed, install specific versions known to be compatible
# !pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Import after installation
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


2.2.1
0.17.1


In [1]:
import anvil.server
import anvil.media
from ultralytics import YOLO  # Adjust this import based on your YOLO version
import os
import cv2  # Import OpenCV
from sklearn.cluster import KMeans
from PIL import Image, ImageOps, UnidentifiedImageError
import torch
import time
import torchvision
from torchvision.transforms import functional as F
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap
import webcolors
from datetime import datetime
import numpy as np

# If needed, install specific versions known to be compatible
# !pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Import after installation
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Define your class names based on the model's classes
CLASS_NAMES = ['Arbequina -Spain-', 'Bella di Cerignola -Italy-', 'Nocellara del Belice -Italy-', 'Picual -Spain-']

# Load the Faster R-CNN model
def load_model(n_classes):
    model = fasterrcnn_mobilenet_v3_large_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, n_classes)
    
    checkpoint = torch.load('/Users/kimberleycollins/Desktop/Thesis/Thesis/Prototype/Application/model_final.pth', map_location='cpu')
    model.load_state_dict(checkpoint)
    
    model.eval()
    return model

# Call the load model function with the correct number of classes
model = load_model(5)

# Connect to Anvil server
anvil.server.connect("server_PKZW2MEMHK7Q5OPOFHBZZSA3-VMYLTPSHUVY2Y7J2")

def find_dominant_color(image):
    try:
        pixels = image.reshape((-1, 3))
        kmeans = KMeans(n_clusters=1, n_init=10)
        kmeans.fit(pixels)
        dominant_color = kmeans.cluster_centers_.astype(int)[0]
        return '#%02x%02x%02x' % tuple(dominant_color)
    except Exception as e:
        print(f"Error finding dominant color: {e}")
        return None

def check_olive_color(hex_color):
    try:
        rgb_color = tuple(int(hex_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
        luminance = 0.299 * rgb_color[0] + 0.587 * rgb_color[1] + 0.114 * rgb_color[2]
        
        color_ranges = {
            "unripe": (120, 180, 70),
            "partially_ripe": (90, 110, 50),
            "fully_ripe": (60, 0, 70),
            "overripe": (20, 20, 20)
        }
        
        ripeness = "unknown"
        if rgb_color >= color_ranges["unripe"] and luminance > 150:
            ripeness = "unripe"
        elif rgb_color >= color_ranges["partially_ripe"] and 100 <= luminance <= 150:
            ripeness = "partially ripe"
        elif rgb_color >= color_ranges["fully_ripe"] and 50 <= luminance <= 100:
            ripeness = "fully ripe"
        elif rgb_color <= color_ranges["overripe"] or luminance < 50:
            ripeness = "overripe"
        
        ripeness_messages = {
            "unripe": "The olives are unripe, ideal for pickling or early harvest products. Not ideal for oil extraction yet.",
            "partially ripe": "The olives are partially ripe and may have a more bitter taste, suitable for certain types of processing. Best for harvesting if aiming for early-harvest olive oil.",
            "fully ripe": "The olives appear to be fully ripe and ready for most culinary uses or oil extraction. Ideal time for harvesting for high-quality olive oil.",
            "overripe": "The olives are overripe; they might be too soft or mushy, suitable only for some types of oil or immediate consumption. Immediate harvesting is recommended to avoid spoilage.",
            "unknown": "It's difficult to determine the ripeness of these olives based on the image provided."
        }
        
        return ripeness_messages[ripeness]
    
    except Exception as e:
        print(f"Error checking olive color: {e}")
        return "An error occurred while analyzing olive color."

def suggest_dishes_for_olives(olive_color_message):
    if 'Black olives' in olive_color_message:
        return ("Ideal for bold-flavored dishes like tapenade, puttanesca sauce, Mediterranean pizza, or as a rich topping for Greek salads. Black olives can also be used to enhance the flavor of roast lamb or as part of a cheese and charcuterie board.")
    else:
        return ("Perfect for lighter fare such as martinis, Greek salads, or paired with cheese and wine. Green olives can also be used in chicken tagine, olive bread, or as a garnish for fish dishes.")

def olive_preservation_tips(color):
    if color == 'green':
        tips = ("Green olives can be preserved in brine, vinegar, or olive oil. "
                "Brining is a traditional method that involves soaking the olives in a salt water solution for several weeks, "
                "allowing them to ferment slightly and develop their flavor. "
                "After brining, they can be seasoned with herbs and stored in olive oil or vinegar for enhanced taste.")
    else:
        tips = ("Black olives are often preserved in oil or dry-cured. "
                "Dry-curing involves covering the olives in salt to draw out moisture, resulting in a wrinkled, concentrated flavor. "
                "After curing, they can be rinsed and stored in olive oil with herbs or lemon zest for added flavor. "
                "Storing in oil helps keep them moist and flavorful.")
    return tips

def olive_health_tips():
    tips = (
        "To keep your preserved olives healthy, ensure they are stored in a cool, dark place, away from direct sunlight. "
        "If preserved in liquid (brine, vinegar, or oil), make sure the olives are fully submerged to prevent mold and oxidation. "
        "Containers should be sealed tightly to keep out air and contaminants. "
        "For olives preserved in oil, use sterilized jars and high-quality olive oil to prevent the growth of bacteria, including botulism. "
        "Regularly check your olives for signs of spoilage, such as off-odors, discoloration, or mold. "
        "If preserved in brine, changing the brine every few months can help maintain freshness and flavor. "
        "Always use clean utensils to handle olives to prevent contamination. "
        "Consider refrigerating after opening to prolong their shelf life, especially for olives in liquid preservations."
    )
    return tips

def generate_pdf_report(filename, olive_name, olive_color, color_name, olive_color_message, preservation_tips, health_tips, dish_suggestion, current_date, current_time):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    def draw_wrapped_text(c, text, x, y, max_width, line_height):
        lines = textwrap.wrap(text, width=max_width)
        for line in lines:
            c.drawString(x, y, line)
            y -= line_height  # Adjust this value for line spacing

    max_width = 100  # Adjust this value to control text wrapping width
    line_height = 12  # Adjust this value for line spacing

    c.setFont("Helvetica", 14)
    c.drawString(30, height - 50, "Olive Analysis Report")
    c.setFont("Helvetica", 12)
    c.drawString(30, height - 80, f"Olive Name: {olive_name}")
    c.drawString(30, height - 100, f"Olive Color: {olive_color} ({color_name})")
    
    y_position = height - 130
    draw_wrapped_text(c, f"Olive Color Message: {olive_color_message}", 30, y_position, max_width, line_height)
    y_position -= line_height * (len(textwrap.wrap(f"Olive Color Message: {olive_color_message}", width=max_width)) + 1)
    
    draw_wrapped_text(c, f"Preservation Tips: {preservation_tips}", 30, y_position, max_width, line_height)
    y_position -= line_height * (len(textwrap.wrap(f"Preservation Tips: {preservation_tips}", width=max_width)) + 1)
    
    draw_wrapped_text(c, f"Health Tips: {health_tips}", 30, y_position, max_width, line_height)
    y_position -= line_height * (len(textwrap.wrap(f"Health Tips: {health_tips}", width=max_width)) + 1)
    
    draw_wrapped_text(c, f"Suggested Dish: {dish_suggestion}", 30, y_position, max_width, line_height)

    # Add the current date and time at the end of the report
    y_position -= 2 * line_height
    c.drawString(30, y_position, f"Report Date: {current_date}")
    y_position -= line_height
    c.drawString(30, y_position, f"Report Time: {current_time}")

    c.save()

def get_color_name(hex_color):
    try:
        rgb_color = webcolors.hex_to_rgb(hex_color)
        color_name = closest_color(rgb_color)
        return color_name
    except Exception as e:
        print(f"Error getting color name: {e}")
        return "Unknown color"

def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color.red) ** 2
        gd = (g_c - requested_color.green) ** 2
        bd = (b_c - requested_color.blue) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

@anvil.server.callable
def classify_image(media):
    try:
        with anvil.media.TempFile(media) as filename:
            image = Image.open(filename)
            if image.mode != 'RGB':
                image = image.convert('RGB')
            
            # Prepare the image for the model
            img_tensor = F.to_tensor(image)
            img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension

            start_time = time.time()  # Start time before model inference
            with torch.no_grad():
                predictions = model(img_tensor)
            inference_time = time.time() - start_time  # Time taken for model to predict
            
            # Debugging prints to check the predictions
            print(f"Predictions: {predictions}")

            if len(predictions[0]['boxes']) == 0:
                return {"error": "No olives detected in the image."}
            
            scores = predictions[0]['scores']
            print(f"Scores: {scores}")

            max_index = scores.argmax().item()
            print(f"Max index: {max_index}")

            bbox = predictions[0]['boxes'][max_index].numpy().astype(int)
            class_index = predictions[0]['labels'][max_index].item()
            olive_name = CLASS_NAMES[class_index]

            print(f"0: {img_tensor.shape[3]}x{img_tensor.shape[2]} 1 {olive_name} , {inference_time * 1000:.1f}ms")
            print(f"Speed: {inference_time * 1000:.1f}ms inference")

            np_image = np.array(image)
            olive_portion = np_image[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            dominant_color = find_dominant_color(olive_portion)

            if dominant_color is None:
                return {"error": "Failed to find a dominant color in the selected portion of the image."}

            # Convert dominant color to RGB and get the closest color name
            rgb_color = webcolors.hex_to_rgb(dominant_color)
            color_name = closest_color(rgb_color)
            olive_color_message = check_olive_color(dominant_color)
            dish_suggestion = suggest_dishes_for_olives(olive_color_message)
            color = 'green' if 'Green' in olive_color_message else 'black'
            preservation_tips = olive_preservation_tips(color)
            health_tips = olive_health_tips()

            # Get current date and time
            current_date = datetime.now().strftime('%A, %B %d, %Y')
            current_time = datetime.now().strftime('%H:%M:%S')

            # Generate PDF report
            pdf_filename = "/tmp/olive_analysis_report.pdf"
            generate_pdf_report(pdf_filename, olive_name, dominant_color, color_name, olive_color_message, preservation_tips, health_tips, dish_suggestion, current_date, current_time)
            pdf_report = anvil.media.from_file(pdf_filename, "application/pdf")

            result = {
                "olive_name": olive_name,
                "timing_message": f"0: {img_tensor.shape[3]}x{img_tensor.shape[2]} 1 {olive_name} , {inference_time * 1000:.1f}ms",
                "speed_message": f"Speed: {inference_time * 1000:.1f}ms inference",
                "dominant_color": dominant_color,
                "color_name": color_name,
                "olive_color_message": olive_color_message,
                "dish_suggestion": dish_suggestion,
                "preservation_tips": preservation_tips,
                "health_tips": health_tips,
                "pdf_report": pdf_report
            }

            print(f"Result: {result}")
            return result

    except Exception as e:
        print(f"Error classifying image: {e}")
        return {"error": f"An unexpected error occurred: {str(e)}"}

@anvil.server.callable
def generate_report(olive_name, olive_color, color_name, olive_color_message, preservation_tips, health_tips, dish_suggestion, current_date, current_time):
    filename = "/tmp/olive_analysis_report.pdf"
    generate_pdf_report(filename, olive_name, olive_color, color_name, olive_color_message, preservation_tips, health_tips, dish_suggestion, current_date, current_time)
    return anvil.media.from_file(filename, "application/pdf")

# Keep the server running
anvil.server.wait_forever()


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
Predictions: [{'boxes': tensor([[160.4899, 643.6084, 179.0468, 662.2588],
        [615.6562, 641.9152, 635.9840, 660.0482],
        [615.6990, 298.9256, 630.3109, 314.4924]]), 'labels': tensor([3, 3, 3]), 'scores': tensor([0.2216, 0.1666, 0.1246])}]
Scores: tensor([0.2216, 0.1666, 0.1246])
Max index: 0
0: 1388x690 1 Picual -Spain- , 336.3ms
Speed: 336.3ms inference
Result: {'olive_name': 'Picual -Spain-', 'timing_message': '0: 1388x690 1 Picual -Spain- , 336.3ms', 'speed_message': 'Speed: 336.3ms inference', 'dominant_color': '#dadada', 'color_name': 'gainsboro', 'olive_color_message': 'The olives are unripe, ideal for pickling or early harvest products. Not ideal for oil extraction yet.', 'dish_suggestion': 'Perfect for lighter fare such as martinis, Greek salads, or paired with cheese and wine. Green olives can also be used in chicken tagine, olive bread, or as a garnish for fish 

KeyboardInterrupt: 